In [1]:
from joblib import load
import numpy as np
import cv2
from skimage.feature import hog, local_binary_pattern as lbp
from PIL import Image

# Load model and scaler
clf = load('random_forest_model.pkl')  
scaler = load('scaler.pkl')

In [2]:
from skimage import exposure
# Function to load image data and convert to numpy array
def convertImgToNumpyArr(image_path):
    try:
        img = Image.open(image_path)  # creating reference variable img to access image data
        img = img.resize((100, 100))  # resize image to 100x100 in case it is not
        img = np.array(img)  # converting image data from JPG to numpy array
        #img = img / 255.0  # normalizing RGB values (cv2 needs 0-255 range)
        return img
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")  # error handling
        return None

# Function to compute colour histogram features for an image

def computeColourHist(image, bins=20):

    # conversion from RGB to HSV
    hsvImg = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    hsvImg = hsvImg / 255.0  # normalizing
    
    # extracting histograms for Hue, Saturation, and Value
    hueHist = np.histogram(hsvImg[:,:,0], bins=bins, range=(0, 1))[0]  
    satHist = np.histogram(hsvImg[:,:,1], bins=bins, range=(0, 1))[0]  
    valHist = np.histogram(hsvImg[:,:,2], bins=bins, range=(0, 1))[0]  
    
    # concatenating the histograms of Hue, Saturation, and Value and returning
    return np.concatenate((hueHist, satHist, valHist))

def computeLBP_Features(image):

     # conversion to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # computing LBP features
    lbpImg = lbp(gray, method="uniform", P = 8, R = 1) 
    
    bins = np.arange(0, 11) # number of bins for histogram (10)
    lbpHist, _ = np.histogram(lbpImg.flatten(), bins=bins, range=(0, 10)) # computing histogram

    return lbpHist / (np.sum(lbpHist) + 1e-6)

def extract_features(image_path):
    img = convertImgToNumpyArr(image_path)
    ch_features = computeColourHist(img)
    lbp_features = computeLBP_Features(img)
    return np.concatenate([ch_features, lbp_features])  # Combine HOG + LBP

In [4]:
# Path to new image
new_image_path = "apple.jpg"  # Change this to your image path

# Extract features (HOG + LBP only)
features = extract_features(new_image_path).reshape(1, -1)  # Reshape to 2D array (1 sample, n features)

# Scale features (use the same scaler!)
scaled_features = scaler.transform(features)

# Predict
predicted_class = clf.predict(scaled_features)[0]
print(f"Predicted class: {predicted_class}")

Predicted class: Apple
